In [144]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
tqdm.pandas()

In [2]:
#help(pd.read_csv)

In [3]:
data1 = pd.read_csv('lst_announcement_data_1.gzip', compression='gzip')
print('data1 has been read')
data2 = pd.read_csv('lst_announcement_data_2.gzip', compression='gzip')
print('data2 has been read')
data3 = pd.read_csv('lst_announcement_data_3.gzip', compression='gzip')
print('data3 has been read')

data1 has been read
data2 has been read
data3 has been read


In [5]:
%%time
data = pd.concat([data1, data2, data3])

CPU times: user 4.74 s, sys: 18.4 s, total: 23.1 s
Wall time: 39.5 s


In [6]:
del data1
del data2
del data3

In [7]:
data.head()

,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,191401467,NaN,townhouseSale,"{""price"":5.5E7,""currency"":""rur"",""priceType"":""a...",Все предложения в Барвихе! ОКП Барвиха Клаб. В...,400.00,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""monolith"",""parking"":{},""floor...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,5.0,NaN,NaN,NaN,True,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
1,212158376,NaN,commercialLandRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...",Сдается от собственника в аренду площадка 560 ...,NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""settlements"",""possibleToChangeStatu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
2,227815832,NaN,landSale,"{""price"":240000.0,""currency"":""rur"",""priceType""...",Продам участок в жилом районе с. ХОМУТОВО,NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
3,238017518,1.0,flatRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...","Привет, наконец ты попал сюда!\n\nУютная студи...",27.00,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""parking"":{},""floorsCo...",{},True,NaN,True,True,NaN,True,True,NaN,NaN,NaN,NaN,euro,False,NaN,True,NaN,NaN,NaN,False,NaN,1.0,NaN,NaN,studio,NaN,2022-08-01
4,241597454,20.0,flatSale,"{""price"":1.6E7,""currency"":""rur"",""priceType"":""a...",Видовой 20 этаж. Корпус Д. Срочная перепродажа...,60.65,"{""countryId"":138,""undergrounds"":[],""calculated...","{""name"":""ЖК «CASABLANKA (Касабланка)»"",""materi...",{},NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,22.0,NaN,False,1.0,1.0,18+20,NaN,rooms,NaN,2022-08-01


### Предыдущие действия по преобразованию данных

In [8]:
def get_nan_cols(df, nan_percent=0.8):
    """
        функция для определения колонок, в которых много NaN
    """
    threshold = len(df.index) * nan_percent
    return [c for c in df.columns if df[c].isna().sum() >= threshold]

colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data.drop(['ptn_dadd'], axis=1, inplace=True) 

In [9]:
data['loc_id'] = data['geo'].progress_apply(lambda x: json.loads(x)['address'][0]['id'])
data['lat'] = data['geo'].progress_apply(lambda x: json.loads(x)['coordinates']['lat'])
data['lng'] = data['geo'].progress_apply(lambda x: json.loads(x)['coordinates']['lng'])
data.drop(['geo'], axis=1, inplace=True)

100%|██████████████████████████████| 2778005/2778005 [01:08<00:00, 40674.63it/s]


In [10]:
data['bargainterms'] = data['bargainterms'].progress_apply(json.loads)
keys_bargainterms = data['bargainterms'].apply(lambda x: x.keys()).explode().unique()
bargainterms_df = pd.DataFrame(data['bargainterms'].values.tolist())

data['building'] = data['building'].progress_apply(json.loads)
keys_building = data['building'].apply(lambda x: x.keys()).explode().unique()
building_df = pd.DataFrame(data['building'].values.tolist())

data['land'] = data['land'].progress_apply(json.loads)
keys_land = data['land'].apply(lambda x: x.keys()).explode().unique()
land_df = pd.DataFrame(data['land'].values.tolist())

land_df.rename(columns={'type': 'land_type'}, inplace=True)

100%|█████████████████████████████| 2778005/2778005 [00:06<00:00, 452488.70it/s]


In [11]:
# удаление строковых колонок
def del_str_cols(df):
    columns = df.columns
    for col in columns:
        if type(df[col].iloc[1]) == str:
            df.drop(col, inplace=True, axis=1)
    return df

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

data = del_str_cols(data)
bargainterms_df = del_str_cols(bargainterms_df)
building_df = del_str_cols(building_df)
land_df = del_str_cols(land_df)

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

(2778005, 30)
(2778005, 28)
(2778005, 33)
(2778005, 5)
(2778005, 28)
(2778005, 22)
(2778005, 33)
(2778005, 3)


In [12]:
# почистим колонки с большим кол-вом пропусков и удалим ненужные словарные
data.drop(['bargainterms', 'building', 'land'], axis=1, inplace=True)
colss = get_nan_cols(data, 0.9)
data.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(bargainterms_df, 0.9)
bargainterms_df.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(building_df, 0.9)
building_df.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(land_df, 0.9)
land_df.drop(colss, axis=1, inplace=True)

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

(2778005, 25)
(2778005, 9)
(2778005, 17)
(2778005, 1)


In [13]:
display(data.head())
display(bargainterms_df.head())
display(building_df.head())
land_df.head()

,id,floornumber,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,loc_id,lat,lng
0,191401467,NaN,400.00,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4593,55.718725,37.272026
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4579,61.809605,34.296402
2,227815832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4572,52.485696,104.341456
3,238017518,1.0,27.00,True,NaN,True,NaN,True,True,NaN,euro,False,True,NaN,NaN,False,NaN,1.0,NaN,NaN,studio,NaN,4553,45.021564,38.925492
4,241597454,20.0,60.65,NaN,2.0,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,22.0,False,1.0,1.0,18+20,NaN,rooms,NaN,4584,43.485547,39.893921


,price,includedOptions,mortgageAllowed,clientFee,agentFee,prepayMonths,deposit,utilitiesTerms,saleType
0,55000000.0,[],True,NaN,NaN,NaN,NaN,NaN,NaN
1,6000.0,[],NaN,0.0,0.0,1.0,NaN,NaN,NaN
2,240000.0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12990.0,[],NaN,0.0,0.0,1.0,8000.0,"{'includedInPrice': False, 'price': 1.0, 'flow...",NaN
4,16000000.0,[],True,NaN,NaN,NaN,NaN,NaN,free


,materialType,parking,floorsCount,heatingType,cranageTypes,extinguishingSystemTypes,liftTypes,infrastructure,totalArea,openingHours,deadline,name,hasGarbageChute,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight
0,monolith,{},4.0,centralGas,[],[],[],{},400.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN
3,brick,{},3.0,NaN,[],[],[],{},27.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN
4,monolith,{'type': 'multilevel'},20.0,NaN,[],[],[],{},60.7,{},{},ЖК «CASABLANKA (Касабланка)»,True,3.0,2014.0,NaN,NaN


,area
0,2.0
1,5.0
2,10.0
3,NaN
4,NaN


### Очистка от выбросов


In [57]:
# here is testing
col_name = building_df['ceilingHeight']
print(col_name.min())
print(col_name.max())

0.0
4500.0


In [56]:
# rows to delete
#data.drop(data[data['kitchenarea'] == -1].index, inplace=True)
#data.drop(data[data['balconiescount'] == -1].index, inplace=True)
#data.drop(data[data['combinedwcscount'] < 0].index, inplace=True) 
#building_df.drop(building_df[building_df['passengerLiftsCount'] < 0].index, inplace=True)
building_df.drop(building_df[building_df['cargoLiftsCount'] < 0].index, inplace=True)

### Разбиение на бины

In [53]:
### test for defining parameter q
col_name = building_df['buildYear']
q =10
pd.qcut(col_name, q=q)

0                        NaN
1                        NaN
2                        NaN
3                        NaN
4           (2011.0, 2015.0]
                 ...        
2778000                  NaN
2778001     (2004.0, 2011.0]
2778002     (1970.0, 1979.0]
2778003    (999.999, 1961.0]
2778004                  NaN
Name: buildYear, Length: 2778004, dtype: category
Categories (10, interval[float64, right]): [(999.999, 1961.0] < (1961.0, 1970.0] < (1970.0, 1979.0] < (1979.0, 1990.0] ... (2011.0, 2015.0] < (2015.0, 2018.0] < (2018.0, 2021.0] < (2021.0, 2029.0]]

In [61]:
def my_qcut(col, q):
    
    tmp = pd.qcut(col, q=q)
    encoder = dict(zip(tmp.unique(), list(range(q + 1))))
    return col.map(encoder)

##### data:

floornumber 9
totalarea 10
hasfurniture BOOL
roomscount 4
haskitchenfurniture	hastv	haswasher	hasbathtub BOOL
bedroomscount 4
repairtype CAT
petsallowed	hasfridge BOOL
wclocationtype CAT
kitchenarea	10
isapartments BOOL
loggiascount 1
balconiescount 3
##allroomsarea DELETE
livingarea 10
flattype CAT
combinedwcscount 2
loc_id - возможно, ранжирование не имеет смысла
lat	10
lng	10

##### bargainterms_df:

price 10
##includedOptions DELETE
mortgageAllowed	BOOL
clientFee 2
agentFee 1
prepayMonths 1
deposit 4	
utilitiesTerms DICT
saleType CAT

##### building_df:

materialType CAT
parking DICT
floorsCount 6
heatingType CAT
##cranageTypes DELETE
##extinguishingSystemTypes DELETE
##liftTypes DELETE
infrastructure DICT
totalArea 10
openingHours DICT
deadline DICT	
##name DELETE
hasGarbageChute BOOL
passengerLiftsCount 2
buildYear 10
cargoLiftsCount 3
ceilingHeight 10

##### land_df:

area 10

In [69]:
data['floornumber'] = my_qcut(data['floornumber'], 9)
data['totalarea'] = my_qcut(data['totalarea'], 10)
data['roomscount'] = my_qcut(data['roomscount'], 4)
data['bedroomscount'] = my_qcut(data['bedroomscount'], 4)
data['kitchenarea'] = my_qcut(data['kitchenarea'], 10)
data['loggiascount'] = my_qcut(data['loggiascount'], 1)
data['balconiescount'] = my_qcut(data['balconiescount'], 2)
data['livingarea'] = my_qcut(data['livingarea'], 10)
data['combinedwcscount'] = my_qcut(data['combinedwcscount'], 2)
data['lat'] = my_qcut(data['lat'], 10)
data['lng'] = my_qcut(data['lng'], 10)

bargainterms_df['price'] = my_qcut(bargainterms_df['price'], 10)
bargainterms_df['clientFee'] = my_qcut(bargainterms_df['clientFee'], 2)
bargainterms_df['agentFee'] = my_qcut(bargainterms_df['agentFee'], 1)
bargainterms_df['prepayMonths'] = my_qcut(bargainterms_df['prepayMonths'], 1)
bargainterms_df['deposit'] = my_qcut(bargainterms_df['deposit'], 4)

building_df['floorsCount'] = my_qcut(building_df['floorsCount'], 6)
building_df['totalArea'] = my_qcut(building_df['totalArea'], 10)
building_df['passengerLiftsCount'] = my_qcut(building_df['passengerLiftsCount'], 2)
building_df['buildYear'] = my_qcut(building_df['buildYear'], 10)
building_df['cargoLiftsCount'] = my_qcut(building_df['cargoLiftsCount'], 2)
building_df['ceilingHeight'] = my_qcut(building_df['ceilingHeight'], 10)

land_df['area'] = my_qcut(land_df['area'], 10)

In [71]:
data = data.join(bargainterms_df).join(building_df).join(land_df)

In [74]:
data

,id,floornumber,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,loc_id,lat,lng,price,includedOptions,mortgageAllowed,clientFee,agentFee,prepayMonths,deposit,utilitiesTerms,saleType,materialType,parking,floorsCount,heatingType,cranageTypes,extinguishingSystemTypes,liftTypes,infrastructure,totalArea,openingHours,deadline,name,hasGarbageChute,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight,area
0,191401467,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4593,0,0.0,0,[],True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,{},0.0,centralGas,[],[],[],{},0.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN,0.0
0,275557007,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,euro,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,studio,1.0,4584,5,3.0,0,[],True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,{},0.0,centralGas,[],[],[],{},0.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN,0.0
0,274482535,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4593,0,5.0,0,[],True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,{},0.0,centralGas,[],[],[],{},0.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4579,1,1.0,1,[],NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,275619810,3.0,5.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,euro,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,rooms,1.0,4629,1,8.0,1,[],NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999997,274206329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4591,8,6.0,2,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN,2.0
999998,275226384,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,False,NaN,NaN,10.06,6.0,studio,NaN,2,1,9.0,5,[],False,NaN,NaN,NaN,NaN,NaN,fz214,NaN,{},6.0,NaN,[],[],[],{},2.0,{},"{'quarter': 'second', 'year': 2024, 'isComplet...",NaN,NaN,NaN,NaN,NaN,1.0,NaN
999998,274265180,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4596,8,4.0,5,[],False,NaN,NaN,NaN,NaN,NaN,fz214,NaN,{},6.0,NaN,[],[],[],{},2.0,{},"{'quarter': 'second', 'year': 2024, 'isComplet...",NaN,NaN,NaN,NaN,NaN,1.0,NaN
999999,275347394,8.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,2.0,NaN,4.0,rooms,NaN,4599,6,8.0,6,[],True,NaN,NaN,NaN,NaN,"{'includedInPrice': False, 'price': 0.0, 'flow...",free,panel,{},5.0,NaN,[],[],[],{},NaN,{},{},NaN,NaN,2.0,NaN,NaN,4.0,NaN


In [78]:
colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data.head()

,id,floornumber,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,loc_id,lat,lng,price,includedOptions,mortgageAllowed,clientFee,agentFee,prepayMonths,deposit,utilitiesTerms,saleType,materialType,parking,floorsCount,heatingType,cranageTypes,extinguishingSystemTypes,liftTypes,infrastructure,totalArea,openingHours,deadline,hasGarbageChute,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight,area,parking_type
0,191401467,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4593,0,0.0,0,[],True,NaN,NaN,NaN,NaN,{},NaN,monolith,{},0.0,centralGas,[],[],[],{},0.0,{},{},NaN,NaN,NaN,NaN,NaN,0.0,NaN
0,275557007,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,euro,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,studio,1.0,4584,5,3.0,0,[],True,NaN,NaN,NaN,NaN,{},NaN,monolith,{},0.0,centralGas,[],[],[],{},0.0,{},{},NaN,NaN,NaN,NaN,NaN,0.0,NaN
0,274482535,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4593,0,5.0,0,[],True,NaN,NaN,NaN,NaN,{},NaN,monolith,{},0.0,centralGas,[],[],[],{},0.0,{},{},NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4579,1,1.0,1,[],NaN,1.0,1.0,1.0,NaN,{},NaN,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,275619810,3.0,5.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,euro,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,rooms,1.0,4629,1,8.0,1,[],NaN,1.0,1.0,1.0,NaN,{},NaN,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [79]:
# удалим нечисловые признаки
data.drop(['hasfurniture', 'haskitchenfurniture', 'hastv', 'haswasher', 'hasbathtub', \
           'repairtype', 'petsallowed', 'hasfridge', 'wclocationtype', 'isapartments', \
           'flattype', 'loc_id', 'includedOptions', 'mortgageAllowed', 'utilitiesTerms', \
           'saleType', 'materialType', 'parking', 'heatingType', 'cranageTypes', \
           'extinguishingSystemTypes', 'liftTypes', 'infrastructure', 'openingHours',\
           'deadline', 'hasGarbageChute', 'allroomsarea'], axis=1, inplace=True)

In [90]:
data.head()

,id,floornumber,totalarea,roomscount,bedroomscount,kitchenarea,loggiascount,balconiescount,allroomsarea,livingarea,combinedwcscount,lat,lng,price,clientFee,agentFee,prepayMonths,deposit,floorsCount,totalArea,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight,area
0,191401467,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
0,275557007,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5,3.0,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
0,274482535,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5.0,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,275619810,3.0,5.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1,8.0,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [106]:
data.shape

(2777998, 25)

### Обработка пропусков

In [115]:
# обработка пропусков
# средним???
def fillna_num(data_col):
    return data_col.fillna(data_col.mean())

for col in tqdm(data.columns[1:]):
    data[col] = fillna_num(data[col])

100%|███████████████████████████████████████████| 23/23 [00:00<00:00, 28.06it/s]


### Группировка по ID

In [121]:
sh1 = data.shape
sh2 = data.drop_duplicates(subset=data.columns[1:], inplace=False).shape
sh1[0] - sh2[0]

340901

In [122]:
data['id'] = data['id'].astype(str) + '+'

In [130]:
[list(data.columns[1:])]

[['floornumber',
  'totalarea',
  'roomscount',
  'bedroomscount',
  'kitchenarea',
  'loggiascount',
  'balconiescount',
  'livingarea',
  'combinedwcscount',
  'lat',
  'lng',
  'price',
  'clientFee',
  'agentFee',
  'prepayMonths',
  'deposit',
  'floorsCount',
  'totalArea',
  'passengerLiftsCount',
  'buildYear',
  'cargoLiftsCount',
  'ceilingHeight',
  'area']]

In [138]:
data_grouped = data.groupby(list(data.columns[1:])).sum().reset_index()
data_grouped

,floornumber,totalarea,roomscount,bedroomscount,kitchenarea,loggiascount,balconiescount,livingarea,combinedwcscount,lat,lng,price,clientFee,agentFee,prepayMonths,deposit,floorsCount,totalArea,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight,area,id
0,1.0,0.0,1.0,2.391695,1.0,1.0,1.000000,1.000000,1.00000,5,4.0,1,1.461308,1.0,1.0,3.000000,3.000000,0.000000,2.00000,3.000000,2.000000,1.000000,5.04352,272724108+
1,1.0,0.0,1.0,2.391695,1.0,1.0,1.000000,1.000000,1.00000,5,6.0,1,1.461308,1.0,1.0,3.000000,6.000000,4.000000,2.00000,5.131124,2.000000,4.417647,5.04352,232387263+
2,1.0,0.0,1.0,2.391695,1.0,1.0,1.000000,3.000000,1.00000,3,3.0,8,1.461308,1.0,1.0,2.522749,3.000000,7.000000,1.79454,9.000000,1.892738,4.417647,10.00000,238805764+
3,1.0,0.0,1.0,2.391695,1.0,1.0,1.000000,3.000000,1.00000,5,1.0,3,1.461308,1.0,1.0,2.522749,3.000000,0.000000,1.79454,4.000000,1.892738,4.417647,10.00000,275812758+
4,1.0,0.0,1.0,2.391695,1.0,1.0,1.000000,3.000000,1.00000,5,1.0,5,1.461308,1.0,1.0,2.522749,2.953382,5.341775,1.79454,5.131124,1.892738,4.417647,10.00000,256615641+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437092,9.0,10.0,3.0,2.391695,10.0,1.0,2.000000,5.000000,1.00000,3,5.0,7,1.461308,1.0,1.0,2.522749,4.000000,10.000000,2.00000,5.131124,2.000000,4.417647,5.04352,275927984+
2437093,9.0,10.0,3.0,2.391695,10.0,1.0,2.000000,10.000000,1.00000,9,8.0,4,1.461308,1.0,1.0,2.522749,5.000000,8.000000,1.79454,5.000000,2.000000,1.000000,5.04352,274350644+
2437094,9.0,10.0,4.0,2.391695,1.0,1.0,1.000000,5.438916,1.00000,4,1.0,7,1.461308,1.0,1.0,2.522749,4.000000,3.000000,1.79454,5.131124,1.892738,4.417647,5.04352,275908886+
2437095,9.0,10.0,4.0,2.391695,1.0,1.0,1.000000,5.438916,1.05114,1,9.0,0,1.461308,1.0,1.0,2.522749,2.000000,0.000000,1.79454,5.131124,1.892738,4.417647,5.04352,274921227+


### Масштабирование данных

In [142]:
data_grouped.columns[:-1]

Index(['floornumber', 'totalarea', 'roomscount', 'bedroomscount',
       'kitchenarea', 'loggiascount', 'balconiescount', 'livingarea',
       'combinedwcscount', 'lat', 'lng', 'price', 'clientFee', 'agentFee',
       'prepayMonths', 'deposit', 'floorsCount', 'totalArea',
       'passengerLiftsCount', 'buildYear', 'cargoLiftsCount', 'ceilingHeight',
       'area'],
      dtype='object')

In [146]:
# отмасштабируем числовые признаки, кроме id
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_grouped[data_grouped.columns[:-1]])
print(np.max(data_scaled), np.min(data_scaled))

1.0 0.0


In [149]:
embeddings = np.hstack((np.array(data_grouped['id']).reshape((data_grouped.shape[0], 1)), data_scaled))
print(embeddings.shape)
embeddings

(2437097, 24)


array([['272724108+', 0.0, 0.0, ..., 1.0, 0.0, 0.5043520465757283],
       ['232387263+', 0.0, 0.0, ..., 1.0, 0.37973852191490226,
        0.5043520465757283],
       ['238805764+', 0.0, 0.0, ..., 0.8927382283214156,
        0.37973852191490226, 1.0],
       ...,
       ['275908886+', 1.0, 1.0, ..., 0.8927382283214156,
        0.37973852191490226, 0.5043520465757283],
       ['274921227+', 1.0, 1.0, ..., 0.8927382283214156,
        0.37973852191490226, 0.5043520465757283],
       ['244882414+', 1.0, 1.0, ..., 0.8927382283214156,
        0.37973852191490226, 0.8]], dtype=object)

In [151]:
# save in binary format
np.save('embeddings_bin_num.npy', embeddings)
np.save('embeddings_bin_num.npy', embeddings[:, 0])  